In [11]:
from sqlalchemy import create_engine, func
from sqlalchemy.orm import sessionmaker
import sys
import os
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../")))
from DB.models import init_db, Circuit, Season, RacingWeekend, Driver, Session, SessionResult, Lap, Team, DriverTeamSession, PitStop
from utils import correct_fuel_effect, extract_driver_strategies, get_base_sector_times, get_tyre_deg_per_driver, get_race_session, get_safety_car_penalty, get_driver_pit_time

import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from itertools import product

# Initialize database connection
global db_session
engine, db_session = init_db()

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', None) 


## 1) Query race into df and pre process

In [12]:

def get_race_df(year, circuit):
	race_session_db = get_race_session(year, circuit, db_session)

	base_sector_times = get_base_sector_times(year, circuit, db_session)

	laps = race_session_db.laps

	# Convert to DataFrame
	laps_data = []
	for lap in laps:
		# Add a row for Sector 1
		laps_data.append({
			"lap_num": lap.lap_num,
			"sector": 1,
			"stint_num": lap.stint_num,
			"stint_lap": lap.stint_lap,
			"position": lap.position,
			"driver_name": lap.driver.driver_name,
			"driver_number": lap.driver.driver_num,
			"sector_time": lap.s1_time,
			"tyre_type": lap.tyre_type,
			"tyre_laps": lap.tyre_laps,
			"pit": lap.pit,
			"pit_time": lap.pit_stop[0].pit_time if lap.pit_stop else None,
			"track_status": lap.track_status,
			"base_sector_time": base_sector_times[lap.driver.driver_num][1]
		})

		# Add a row for Sector 2
		laps_data.append({
			"lap_num": lap.lap_num,
			"sector": 2,
			"stint_num": lap.stint_num,
			"stint_lap": lap.stint_lap,
			"position": lap.position,
			"driver_name": lap.driver.driver_name,
			"driver_number": lap.driver.driver_num,
			"sector_time": lap.s2_time,
			"tyre_type": lap.tyre_type,
			"tyre_laps": lap.tyre_laps,
			"pit": lap.pit,
			"pit_time": lap.pit_stop[0].pit_time if lap.pit_stop else None,
			"track_status": lap.track_status,
			"base_sector_time": base_sector_times[lap.driver.driver_num][2]
		})

		# Add a row for Sector 3
		laps_data.append({
			"lap_num": lap.lap_num,
			"sector": 3,
			"stint_num": lap.stint_num,
			"stint_lap": lap.stint_lap,
			"position": lap.position,
			"driver_name": lap.driver.driver_name,
			"driver_number": lap.driver.driver_num,
			"sector_time": lap.s3_time,
			"tyre_type": lap.tyre_type,
			"tyre_laps": lap.tyre_laps,
			"pit": lap.pit,
			"pit_time": lap.pit_stop[0].pit_time if lap.pit_stop else None,
			"track_status": lap.track_status,
			"base_sector_time": base_sector_times[lap.driver.driver_num][3]
		})

	# Create a DataFrame from the list of dictionaries
	df = pd.DataFrame(laps_data)
	

	df = df.sort_values(["lap_num", "sector", "position"]).reset_index(drop=True)

	return df

	
def add_race_data(df):
	# Calculate cumulative race time for each driver
	df["cumulative_time"] = df.groupby("driver_name")["sector_time"].cumsum()
 
	# Calculate rolling pace (average lap time over the last 5 laps)
	df["pace"] = (
		df.groupby(["driver_name", "sector"])["sector_time"]
		.rolling(window=5, min_periods=1)
		.mean()
		.reset_index(level=[0, 1], drop=True)
	)

	# Get car ahead"s cumulative time (car immediately ahead in position for each lap)
	df["front_cumulative_time"] = df.groupby(["lap_num", "sector"])["cumulative_time"].shift(1)
	# This gap is calculated only for drivers who are not in the lead position (position > 1)
	df["gap"] = df["cumulative_time"] - df["front_cumulative_time"]
	df["gap"] = df["gap"].fillna(0)  # Leader has no car ahead, so gap is 0

	# Calculate tyre difference (compared to car immediately ahead in THIS Sector)
	df["front_tyre"] = df.groupby(["lap_num", "sector"])["tyre_type"].shift(1)
	df["tyre_diff"] = df["front_tyre"] - df["tyre_type"]
	df["tyre_diff"] = df["tyre_diff"].fillna(0)  # Leader has no car ahead

	# Calculate tyre age difference (compared to car immediately ahead in THIS Sector)
	df["front_laps"] = df.groupby(["lap_num", "sector"])["stint_lap"].shift(1)
	df["stint_laps_diff"] = df["front_laps"] - df["stint_lap"]
	df["stint_laps_diff"] = df["stint_laps_diff"].fillna(0)  # Leader has no car ahead

	# Calculate DRS availability (within 1s of car ahead IN THIS Sector)
	df["drs_available"] = (
		(df["gap"] <= 1) &
		(df["position"] > 1) &
		(df["lap_num"] > 1)
	)


	# df["next_position"] = df.groupby("driver_name")["position"].shift(1) 
	# df["overtaken"] = ((df["next_position"] < df["position"]) | 
	# 				  (df["next_position"].isna()))


	# Shift the "position" and "pit" columns
	df["next_position"] = df.groupby("driver_name")["position"].shift(1)
	df["next_pit"] = df.groupby("driver_name")["pit"].shift(-1)

	# Handle NaN values in 'next_pit' by filling them with False
	df["next_pit"] = df["next_pit"].fillna(False)

	# Define the "overtaken" column
	df["overtaken"] = (
		((df["next_position"] < df["position"]) | (df["next_position"].isna()))  # Original condition
		& (~df["next_pit"])  # Ensure the driver in the next position is not pitting
	)

	# # Create target variable for overtaking model)
	# # Step 1: Detect position improvement on the next sector
	# df.loc[:, "next_position"] = df.groupby("driver_name")["position"].shift(-1)
	# df.loc[:, "position_improved"] = (
	# 	(df["next_position"] < df["position"]) |  # Position improved
	# 	(df["next_position"].isna())  # Handle NaN values
	# )

	# # Step 2: Identify the sector with the minimum gap for each driver and lap
	# df.loc[:, "min_gap_sector"] = df[df["position_improved"]].groupby(["driver_name", "lap_num"])["gap"].transform("idxmin")

	# # Step 3: Propagate the overtaken flag to the sector with the minimum gap
	# df.loc[:, "overtaken"] = False  # Initialize the overtake flag
	# for idx in df[df["position_improved"]].index:
	# 	driver = df.loc[idx, "driver_name"]
	# 	lap = df.loc[idx, "lap_num"]
		
	# 	# Find the sector with the minimum gap for this driver and lap
	# 	min_gap_idx = df.loc[
	# 		(df["driver_name"] == driver) & 
	# 		(df["lap_num"] == lap) & 
	# 		(df["position_improved"]), "min_gap_sector"
	# 	].iloc[0]
		
	# 	# Set the overtaken flag only for the sector with the minimum gap
	# 	if idx == min_gap_idx:
	# 		df.loc[idx, "overtaken"] = True

	# # Cleanup intermediate columns
	# df = df.drop(columns=["position_improved", "min_gap_sector"], errors="ignore")
	

	# Cleanup and final sorting
	df = df.drop(columns=["front_cumulative_time", "front_tyre", "next_position"])
	# df = df.sort_values(["lap_num", "sector", "position"]).reset_index(drop=True)

	try:
		new_order = [
			"lap_num", "sector", "base_sector_time", "stint_num", "stint_lap", "position", "driver_name",
			"driver_number", "sector_time", "gap", "cumulative_time", "tyre_type", "tyre_laps", 
			"pit", "pit_time", "drs_available", "overtaken", "tyre_diff", "front_laps", "stint_laps_diff", "track_status", "pace"
		]

		df = df[new_order]
	except:
		print("Extra column")
	return df

global year
global circuit
year = 2024
# circuit = "Melbourne"
circuit = "Sakhir"
# circuit = "Monza"
# circuit = "Mexico City"
# circuit = "Austin"
# circuit = "Yas Island"

race_session_db = get_race_session(year, circuit, db_session)

df = get_race_df(year, circuit)
df = add_race_data(df)


# df[["lap_num", "sector", "position", "driver_name", "gap", "overtaken"]]
# df[df["driver_number"]==14]
# df[df["track_status"]!=1]


/tmp/ipykernel_394056/2289731626.py:120: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["next_pit"] = df["next_pit"].fillna(False)


In [13]:
def setup_race_data(race_df, race_session_db):
	driver_strategies = extract_driver_strategies(race_df)
	
	# Extract tyre degradation curves
	driver_tyre_coefficients = get_tyre_deg_per_driver(race_df)
	
	# Precompute driver strategies

	# Correct fuel effects in the race data
	max_laps = race_df["lap_num"].max()
	race_df = correct_fuel_effect(race_df)

	drivers = race_df["driver_number"].unique()

	base_sector_times = get_base_sector_times(year, circuit, db_session)

	driver_names = {
		driver: race_df[race_df["driver_number"] == driver]["driver_name"].iloc[0]
		for driver in drivers
	}

	# Precompute fuel corrections
	max_fuel_kg = 110
	fuel_effect_per_kg = 0.03
	fuel_corrections = {
		lap: (max_fuel_kg - (lap - 1) * (max_fuel_kg / max_laps)) * fuel_effect_per_kg
		for lap in range(1, max_laps + 1)
	}


	# 
	session_results = (
		db_session.query(SessionResult.grid_pos, Driver.driver_num, SessionResult.end_status)
		.join(Session, Session.session_id == SessionResult.session_id)
		.join(Driver, Driver.driver_id == SessionResult.driver_id)
		.filter(SessionResult.session_id == race_session_db.session_id)
		.all()
	)
	# Convert session results to a dictionary
	starting_positions = {driver_num: grid_pos for grid_pos, driver_num, _ in session_results}

	# Initialize an empty dictionary to store retirements by lap
	retirements_by_lap = {}

	# Iterate through session results to determine retirements
	for driver_id, driver_num, end_status in session_results:
		# Check if the driver retired (end_status is not "Finished" or "+1 Lap")
		if end_status and not (end_status.startswith("Finished") or end_status.startswith("+")):
			# Find the maximum lap number for the driver (last recorded lap)
			lap_retired = df[df["driver_number"] == driver_num]["lap_num"].max()
			
			# Add the driver to the list of retirees for the corresponding lap
			if lap_retired not in retirements_by_lap:
				retirements_by_lap[lap_retired] = []
			retirements_by_lap[lap_retired].append(driver_num)

	# Get initial positions (starting grid positions) for each driver

	df["starting_position"] = None  # Initialize column with None
	for driver_num, grid_pos in starting_positions.items():
		# Find the first occurrence of the driver
		first_row_index = df[df["driver_number"] == driver_num].index[0]
		df.at[first_row_index, "starting_position"] = grid_pos
	
	# Get safety car laps
	safety_car_laps = race_df[(race_df["track_status"] != 1) & (race_df["position"]==1)]["lap_num"].unique().tolist()
	
	# # Group by lap_num and convert each group to a list of dictionaries
	# safety_car_laps = []
	# for lap_num, group in safety_car_laps.groupby("lap_num"):
	# 	safety_car_laps.append(lap_num)
	
	# get how much slower safety car laps are on this track
	penalty_percentage = get_safety_car_penalty(year, circuit, db_session)

	# get pit stop times
	driver_pit_times = get_driver_pit_time(year, circuit, db_session)
	return {
		"driver_tyre_coefficients": driver_tyre_coefficients,
		"driver_strategies": driver_strategies,
		"race_df": race_df,
		"max_laps": max_laps,
		"drivers": drivers,
		"driver_names": driver_names,
		"initial_positions": starting_positions,
		"base_sector_times": base_sector_times,
		"fuel_corrections": fuel_corrections,
		"safety_car_laps": safety_car_laps,
		"retirement_laps": retirements_by_lap,
		"safety_car_penalty_percentage": penalty_percentage,
		"driver_pit_times": driver_pit_times
	}

### Now create overtaking model

In [14]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.utils.class_weight import compute_sample_weight
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report

import numpy as np

# Combine training data from multiple years
years = [2024]
df_training = pd.DataFrame()
for training_year in years:	
    # Get race DataFrame
    df_overtake_training = get_race_df(training_year, circuit)
    # Add additional race data
    df_overtake_training = add_race_data(df_overtake_training)
    # Append to the combined training DataFrame
    df_training = pd.concat([df_training, df_overtake_training], ignore_index=True)

# Define features and target
features = [
    "gap",
    "sector",
    "tyre_diff",
    "stint_laps_diff",
    "drs_available",
    "cumulative_time",
    "sector_time",
    "pace",
    "pit"
]

# Prepare feature matrix (X) and target vector (y)
X = df_training[features].values  # Convert to NumPy array immediately
y = df_training["overtaken"].values

# Handle missing values using mean imputation
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

# Resample the data using SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Train the GradientBoostingClassifier
gbc = GradientBoostingClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=3,
    subsample=0.8,
    random_state=42
)

# Calibrate for better probabilities
model = CalibratedClassifierCV(gbc, method="sigmoid", cv=3)
model.fit(X_resampled, y_resampled)

# Compute feature means for handling missing values during prediction
feature_means = X.mean(axis=0)  # Mean of each feature column

def predict_overtake(data):
    """
    Predict overtakes for a NumPy array of feature data.
    
    Args:
        data (np.ndarray): A 2D NumPy array where each row represents a sample and columns correspond to features.
    
    Returns:
        np.ndarray: Predictions for each sample.
    """
    # Ensure all features are numeric
    data = np.array([
        [int(x) if isinstance(x, bool) else x for x in row]  # Convert booleans to integers
        for row in data
    ], dtype=float)  # Ensure the array is of type float

    # Handle missing values by filling with feature means
    data_filled = np.where(np.isnan(data), feature_means, data)
    
    # Make predictions using the trained model
    predictions = model.predict(data_filled)
    return predictions

# Test the model on the original dataset
# Extract test data as a NumPy array
X_test = df_training[features].values
y_test = df_training["overtaken"].values

# Predict overtakes
predicted_overtakes = predict_overtake(X_test)

# Evaluate performance

accuracy = accuracy_score(y_test, predicted_overtakes)
print(f"Accuracy: {accuracy:.3f}")

# Generate classification report
print("\nClassification Report:")
print(classification_report(
    y_test,
    predicted_overtakes,
    target_names=["No Overtake", "Overtaken"]
))

# Optional: Correlation analysis (if needed)
# correlation_with_target = np.corrcoef(X.T, y)[len(features)]
# print("Correlation with Target Variable:")
# print(correlation_with_target)

/tmp/ipykernel_394056/2289731626.py:120: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["next_pit"] = df["next_pit"].fillna(False)


Accuracy: 0.991

Classification Report:
              precision    recall  f1-score   support

 No Overtake       1.00      0.99      1.00      3329
   Overtaken       0.68      0.86      0.76        58

    accuracy                           0.99      3387
   macro avg       0.84      0.93      0.88      3387
weighted avg       0.99      0.99      0.99      3387



In [15]:
race_data = setup_race_data(df, race_session_db)

def extract_features(active_drivers):
    """
    Extracts features from the active_drivers list of dictionaries and returns a NumPy array.
    
    Args:
        active_drivers (list): List of dictionaries containing driver data.
    
    Returns:
        np.ndarray: A 2D NumPy array where each row corresponds to a driver and columns correspond to features.
    """
    # Define the feature names (adjust based on your model's requirements)
    feature_names = [
    "gap",
    "sector",
    "tyre_diff",
    "stint_laps_diff",
    "drs_available",
    "cumulative_time",
    "sector_time",
    "pace",
    "pit"
    ]

    
    # Extract features for each driver
    features = []
    for driver in active_drivers:
        driver_features = [driver[feature] for feature in feature_names]
        features.append(driver_features)
    
    # Convert to a NumPy array
    return np.array(features, dtype=float)

In [16]:
from collections import deque

def race_sim(precomputed_data, given_driver=None, simulated_strategy=None):
	"""
	simulated_strategy
	"""
	# Extract precomputed data
	driver_tyre_coefficients = precomputed_data["driver_tyre_coefficients"]
	driver_strategies = precomputed_data["driver_strategies"]
	max_laps = precomputed_data["max_laps"]
	drivers = precomputed_data["drivers"]
	driver_names = precomputed_data["driver_names"]
	initial_positions = precomputed_data["initial_positions"]
	base_sector_times = precomputed_data["base_sector_times"]
	fuel_corrections = precomputed_data["fuel_corrections"]
	safety_car_laps = precomputed_data["safety_car_laps"]
	retirement_laps = precomputed_data["retirement_laps"]
	safety_car_penalty_percentage = precomputed_data["safety_car_penalty_percentage"]
	driver_pit_times = precomputed_data["driver_pit_times"]

	print(driver_tyre_coefficients)
	if given_driver and simulated_strategy:
		driver_strategies[given_driver] = simulated_strategy

	drivers_data = []

	for driver in drivers:
		drivers_data.append({
			"driver_number": driver,
			"driver_name": driver_names[driver],
			"pit_schedule": {key: value for key, value in driver_strategies[driver].items() if key != 1},
			"tyre_type": driver_strategies[driver][1],
			"lap_num": 0,
			"sector": 0,
			"sector_time": 0.0,
			"stint_lap": 0,
			"cumulative_time": 0.0,
			"gap": 0.0,
			"pit": False,
			"pace": 0,
			"position": initial_positions[driver],
			"starting_pos": initial_positions[driver],
			"base_sector_times": base_sector_times[driver],
			"tyre_diff": 0,  # Initialize tyre difference as 0
			"stint_laps_diff": 0,  # Initialize stint laps difference as 0
			"drs_available": False,  # Initialize DRS availability as False
			"retired": False,
		})
		
	# {driver: {sector: {last_5: []}}}
	driver_pace_per_sec = {
		driver: {sector: deque(maxlen=5) for sector in range(1, 4)}  # Sectors 1, 2, 3
		for driver in drivers
	}
	

	num_overtakes = 0
	for lap in range(1, max_laps + 1):
		for d in drivers_data:
			d["lap_num"] += 1
			d["stint_lap"] += 1

		saftey_car = lap in safety_car_laps

		if lap in retirement_laps:
			retiring_drivers = retirement_laps[lap]
			
			# Move all drivers behind the retiring drivers up by 1 position
			for driver in retiring_drivers:
				retiring_position = next(
					d["position"] for d in drivers_data if d["driver_number"] == driver
				)
		
				for d in drivers_data:
					if d["position"] > retiring_position:
						d["position"] -= 1

			# Mark whoever has retired as 999 (means not included in overtakes and such)
			for d in drivers_data:
				if d["driver_number"] in retiring_drivers:
					d["retired"] = True
					d["position"] = 999
	
		# Now process each sector
		for sector in range(1, 4):
			for d in drivers_data:
				# If the driver is retired, we just want to skip them
				if d["retired"]:
					continue
				d["sector"] = sector
				# Calculate sector time based on the tyre deg models calcualted before
				a, b, c = driver_tyre_coefficients[d["driver_number"]][d["tyre_type"]][sector]
				sector_time = (
					d["base_sector_times"][sector]  # Base sector time for specific driver
					+ (a * d["stint_lap"]**2 + b * d["stint_lap"] + c)  # Tyre degradation
					+ fuel_corrections[lap]  # Fuel effect
				)

				# Add the percent diff if its a safety car lap (slower under safety car)
				if saftey_car:
					sector_time *= safety_car_penalty_percentage
				
				# Update sector time and cumulative time
				d["sector_time"] = sector_time
				d["cumulative_time"] += sector_time

				# Add to find rolling average/pace later
				driver_pace_per_sec[d["driver_number"]][sector].append(sector_time)

				# Handle pit stops at the start of a lap (sector 1)
				if sector == 1 and lap in d["pit_schedule"]:
					d["pit"] = True  # Mark pit stop
					d["cumulative_time"] += driver_pit_times[d["driver_number"]]  # Add pit stop time
					d["stint_lap"] = 1  # Reset stint lap
					d["tyre_type"] = d["pit_schedule"][lap]  # Change tyre

				else:
					d["pit"] = False
			
			# Re-sort drivers by cumulative time and update positions (as may have changed due to drivers pitting)
			active_drivers = [d for d in drivers_data if not d["retired"]]
			active_drivers.sort(key=lambda x: x["cumulative_time"])
			for i, d in enumerate(active_drivers):
				d["position"] = i + 1

			# Handle overtakes, if we have a safety car then we cant overtake
			if saftey_car:
				continue
			
			# Now loop over again now all the overtakes are calculated
			for d in drivers_data:
				if d["retired"]:
					continue
			
				ahead_pos = d["position"] - 1

				# Now if the driver has someone infront of them (Not P1)
				if ahead_pos > 0:
					# Find ahead driver
					ahead_driver = next(
						a_d for a_d in active_drivers if a_d["position"] == ahead_pos
					)
					# Get the total times
					current_driver_time = d["cumulative_time"] 
					ahead_driver_time = ahead_driver["cumulative_time"]

					# If for some reason the behind driver has a lower total time, then fix (lower is better/faster)
					if ahead_driver_time > current_driver_time:
						# Give the ahead driver the behind drivers time -0.5
						new_ahead_time = current_driver_time - 1
						ahead_driver["cumulative_time"] = new_ahead_time

						ahead_driver_time = new_ahead_time
					
					# calcualte the gap now, incase its different has the total times have changed
					gap = current_driver_time - ahead_driver_time

					d["gap"] = gap

					# if gap < 1:

					# Calculate pace - this is over the last 5 laps in that specific sector for that driver
					sector_times = driver_pace_per_sec[d["driver_number"]][sector]
					if len(sector_times) > 0:
						# find average
						rolling_pace = sum(sector_times) / len(sector_times)
					else:
						rolling_pace = 0.0  # Default value if no sector times are available yet

					
					d["pace"] = rolling_pace
					
					# Tyre difference
					d["tyre_diff"] = ahead_driver["tyre_type"] - d["tyre_type"]
					# Stint laps difference
					d["stint_laps_diff"] = ahead_driver["stint_lap"] - d["stint_lap"]
					# DRS availability
					d["drs_available"] = True

				else:
					d["gap"] = 0

					


			
			active_drivers = [d for d in drivers_data if not d["retired"]]
			# Extract features for prediction
			X_test = extract_features(active_drivers)

			# Predict overtakes
			predicted_overtakes = predict_overtake(X_test)
			for i, driver in enumerate(active_drivers):
				driver["predicted_overtake"] = predicted_overtakes[i]


			for driver in active_drivers:
				if driver["retired"]:
					continue

				ahead_pos = driver["position"] - 1
				if driver["gap"] < 1 and ahead_pos > 0:
					# Find the driver ahead
					ahead_driver = next(d for d in active_drivers if d["position"] == ahead_pos)

					# Check if an overtake is predicted
					if driver["predicted_overtake"]:
						num_overtakes += 1

						# Swap positions
						driver["position"], ahead_driver["position"] = ahead_driver["position"], driver["position"]

						# Swap cumulative times
						driver["cumulative_time"], ahead_driver["cumulative_time"] = (
							ahead_driver["cumulative_time"] - 1, driver["cumulative_time"]
						)

					
	
	# drivers_df.drop(columns=["base_sector_times"], inplace=True)
	print(f"Number of overtakes: {num_overtakes}")

	return drivers_data

drivers_data = race_sim(race_data)
sim_df = pd.DataFrame(drivers_data)
sim_df = sim_df.sort_values(by="position", ascending=True).reset_index(drop=True)
sim_df


{np.int64(1): {np.int64(1): {np.int64(1): [0.0009178771155909023, -0.0033808734085401214, 1.4812535875778954], np.int64(2): [0.00025020590751303367, 0.04678985440191217, 2.3323948143526527], np.int64(3): [0.0003769096949929787, 0.03620968255451015, 0.2515056771404921]}, np.int64(3): {np.int64(1): [0.0015917968379534134, -0.0024937994940423523, 1.3380719741247395], np.int64(2): [0.00060309742596996, 0.04449313587652371, 2.2154982890751547], np.int64(3): [0.0011323805205723763, 0.01480476070882824, 0.3366559121434837]}, 2: {}}, np.int64(3): {np.int64(1): {np.int64(1): [0.005293122930733136, -0.07420533349290309, 1.536182530937377], np.int64(2): [0.001979200966503208, 0.0056913308661828465, 2.64661407244786], np.int64(3): [0.00034201746775766595, 0.03283228341506487, 0.559494751449386]}, np.int64(3): {np.int64(1): [0.00011232320853058154, 0.02678964580874667, 1.189340916540794], np.int64(2): [0.0003753086739474097, 0.04439651935664386, 2.724153121192004], np.int64(3): [-7.776567276396329e

,driver_number,driver_name,pit_schedule,tyre_type,lap_num,sector,sector_time,stint_lap,cumulative_time,gap,pit,pace,position,starting_pos,base_sector_times,tyre_diff,stint_laps_diff,drs_available,retired,predicted_overtake
0,1,Max Verstappen,"{18: 3, 38: 1}",1,57,3,23.558358,20,5758.563644,0.000000,False,41.760063,1,1,"{1: 28.509, 2: 38.269, 3: 22.374}",0,1,True,False,False
1,11,Sergio Perez,"{13: 3, 37: 1}",1,57,3,23.663643,21,5764.461612,5.897968,False,23.682958,2,5,"{1: 28.738, 2: 38.323, 3: 22.41}",0,-1,True,False,False
2,55,Carlos Sainz,"{15: 3, 36: 3}",3,57,3,23.571127,22,5772.060809,7.599197,False,23.575942,3,4,"{1: 28.85, 2: 38.298, 3: 22.337}",-2,-1,True,False,False
3,63,George Russell,"{12: 3, 32: 3}",3,57,3,23.719205,26,5795.556048,23.495239,False,23.734281,4,3,"{1: 28.8, 2: 38.421, 3: 22.264}",0,-4,True,False,False
4,16,Charles Leclerc,"{12: 3, 35: 3}",3,57,3,23.592004,23,5797.586494,2.030446,False,23.649008,5,2,"{1: 28.762, 2: 38.059, 3: 22.263}",0,3,True,False,False
5,44,Lewis Hamilton,"{13: 3, 34: 3}",3,57,3,23.715420,24,5805.109995,7.523500,False,23.735370,6,9,"{1: 28.803, 2: 38.422, 3: 22.369}",0,-1,True,False,False
6,4,Lando Norris,"{14: 3, 34: 3}",3,57,3,23.849990,24,5820.889669,15.779675,False,23.883323,7,7,"{1: 28.836, 2: 38.207, 3: 22.362}",0,0,True,False,False
7,18,Lance Stroll,"{10: 3, 28: 3}",3,57,3,24.084792,30,5821.120713,0.231044,False,24.125986,8,12,"{1: 28.65, 2: 38.609, 3: 22.694}",0,-6,True,False,False
8,81,Oscar Piastri,"{13: 3, 35: 3}",3,57,3,23.767863,23,5825.053962,3.933249,False,23.811072,9,8,"{1: 28.901, 2: 38.343, 3: 22.439}",0,7,True,False,False
9,24,Guanyu Zhou,"{10: 3, 29: 3}",3,57,3,24.011421,29,5844.731755,19.677793,False,24.063454,10,17,"{1: 29.154, 2: 38.944, 3: 22.658}",0,-6,True,False,False


In [17]:
import pandas as pd
import numpy as np

def compare_simulation_with_actual(sim_df, race_df):
	# Initialize a list to store comparison results
	comparison_results = []
	
	# Get unique drivers from the simulated DataFrame
	drivers = sim_df["driver_number"].unique()
	
	# Calculate cumulative times for each driver in both simulated and actual data
	for driver in drivers:
		# Simulated cumulative time for the driver
		sim_cumulative_time = sim_df[sim_df["driver_number"] == driver]["cumulative_time"].max()
		
		# Actual cumulative time for the driver
		actual_cumulative_time = race_df[race_df["driver_number"] == driver]["sector_time"].sum()

		# if actual_cumulative_time < 5000:
		# 	continue
		# Calculate the absolute error for the driver
		absolute_error = abs(sim_cumulative_time - actual_cumulative_time)
		
		# Store the results for the driver
		comparison_results.append({
			"driver_number": driver,
			"simulated_cumulative_time": sim_cumulative_time,
			"actual_cumulative_time": actual_cumulative_time,
			"absolute_error": absolute_error
		})
	
	# Convert the results to a DataFrame
	comparison_df = pd.DataFrame(comparison_results)
	
	# Calculate the total Mean Absolute Error (MAE)
	total_mae = comparison_df["absolute_error"].mean()
	
	return comparison_df, total_mae


# Example usage
# Assuming `sim_df` is the output of your simulation and `df` is the actual race DataFrame
comparison_df, total_mae = compare_simulation_with_actual(sim_df, df)



# Print the total MAE
print("\Mean Absolute Error (MAE):", total_mae)
comparison_df

\Mean Absolute Error (MAE): 330.0766763138094


,driver_number,simulated_cumulative_time,actual_cumulative_time,absolute_error
0,1,5758.563644,5472.340,286.223644
1,11,5764.461612,5493.612,270.849612
2,55,5772.060809,5495.858,276.202809
3,63,5795.556048,5518.095,277.461048
4,16,5797.586494,5511.324,286.262494
5,44,5805.109995,5519.946,285.163995
6,4,5820.889669,5518.585,302.304669
7,18,5821.120713,5556.481,264.639713
8,81,5825.053962,5526.014,299.039962
9,24,5844.731755,5474.570,370.161755


In [18]:
def get_accuracy(year, circuit, sim_df):
	# Fetch the actual race session from the database
	race_session = (
		db_session.query(Session)
		.join(RacingWeekend, Session.weekend_id == RacingWeekend.racing_weekend_id)
		.join(Circuit, RacingWeekend.circuit_id == Circuit.circuit_id)
		.filter(
			RacingWeekend.year == year,
			Circuit.circuit_name == circuit,
			Session.session_type == "Race"
		)
		.first()
	)

	if not race_session:
		raise ValueError(f"No race session found for {year} at {circuit}")

	# Fetch the actual race laps and results
	laps = race_session.laps
	session_results = (
		db_session.query(SessionResult.position, Driver.driver_num)
		.join(Session, Session.session_id == SessionResult.session_id)
		.join(Driver, Driver.driver_id == SessionResult.driver_id)
		.filter(SessionResult.session_id == race_session.session_id)
		.all()
	)

	# Convert session_results to a dictionary for easy lookup
	actual_results = {driver_num: position for position, driver_num in session_results}

	# Extract simulated results from sim_df
	sim_df = sim_df[~sim_df["retired"]]
	sim_results = sim_df.groupby("driver_number").last()["position"].to_dict()

	# Ensure both results have the same drivers
	common_drivers = set(actual_results.keys()).intersection(sim_results.keys())
	if not common_drivers:
		raise ValueError("No common drivers found between actual and simulated results")

	# Filter results to only include common drivers
	actual_positions = [actual_results[driver] for driver in common_drivers]
	sim_positions = [sim_results[driver] for driver in common_drivers]

	# Calculate accuracy metrics
	position_accuracy = sum(1 for a, s in zip(actual_positions, sim_positions) if a == s) / len(common_drivers)
	top_3_accuracy = sum(1 for a, s in zip(actual_positions, sim_positions) if (a <= 3 and s <= 3)) / 3
	mean_error = sum(abs(a - s) for a, s in zip(actual_positions, sim_positions)) / len(common_drivers)
	total_error = sum(abs(a - s) for a, s in zip(actual_positions, sim_positions))

	# Return accuracy metrics
	return {
		"position_accuracy": position_accuracy,
		"top_3_accuracy": top_3_accuracy,
		"mean_error": mean_error,
		"total_error": total_error,
	}

get_accuracy(year, circuit, sim_df)

{'position_accuracy': 0.3,
 'top_3_accuracy': 1.0,
 'mean_error': 1.2,
 'total_error': 24}

In [19]:
from skopt import gp_minimize
from skopt.space import Integer, Categorical

def optimize_strategy_up_to_3_stops(precomputed_data, driver_number=14, n_calls=100):

	max_laps = precomputed_data["max_laps"]

	# Define the search space
	space = [
		Categorical([1, 2, 3], name='start_tyre'),       # Starting tyre (1=Hard, 2=Medium, 3=Soft)
		Integer(2, max_laps - 1, name='pit1_lap'),      # First pit stop lap (optional)
		Categorical([1, 2, 3], name='pit1_tyre'),        # First pit stop tyre (optional)
		Integer(2, max_laps - 1, name='pit2_lap'),      # Second pit stop lap (optional)
		Categorical([1, 2, 3], name='pit2_tyre'),        # Second pit stop tyre (optional)
		Integer(2, max_laps - 1, name='pit3_lap'),      # Third pit stop lap (optional)
		Categorical([1, 2, 3], name='pit3_tyre'),        # Third pit stop tyre (optional)
	]

	def objective(params):
		start_tyre, pit1_lap, pit1_tyre, pit2_lap, pit2_tyre, pit3_lap, pit3_tyre = params

		# Construct the strategy dictionary
		strategy = {1: start_tyre}  # Always include the starting tyre

		# Add pit stops only if their laps are valid and in ascending order
		pit_laps = sorted(set([pit1_lap, pit2_lap, pit3_lap]))
		tyres = [pit1_tyre, pit2_tyre, pit3_tyre]

		# Ensure pit laps are within bounds and in ascending order
		valid_pit_laps = []
		valid_tyres = []
		for lap, tyre in zip(pit_laps, tyres):
			if 2 <= lap < max_laps:  # Only include valid pit laps
				valid_pit_laps.append(lap)
				valid_tyres.append(tyre)

		# Add valid pit stops to the strategy
		for lap, tyre in zip(valid_pit_laps, valid_tyres):
			strategy[lap] = tyre

		# Penalize strategies with fewer than 2 distinct tyre types
		tyre_types_used = set(strategy.values())
		if len(tyre_types_used) < 2:
			return 20.0  # Penalize with worst position

		# Run the simulation
		try:
			drivers_df = race_sim(precomputed_data, given_driver=driver_number, simulated_strategy=strategy)
			final_position = drivers_df[drivers_df['driver_number'] == driver_number]['position'].iloc[-1]
			return final_position  # Minimize finishing position
		except Exception as e:
			print(f"Error during simulation: {e}")
			return 20.0  # Return worst position in case of errors

	# Perform Bayesian optimization
	result = gp_minimize(
		objective,
		space,
		n_calls=n_calls,
		random_state=42,
		verbose=True
	)

	# Extract best parameters and construct the optimal strategy
	best_params = result.x
	best_strategy = {1: best_params[0]}  # Always include the starting tyre

	# Add valid pit stops to the strategy
	pit_laps = sorted(set([best_params[1], best_params[3], best_params[5]]))
	tyres = [best_params[2], best_params[4], best_params[6]]

	for lap, tyre in zip(pit_laps, tyres):
		if 2 <= lap < precomputed_data["max_laps"]:
			best_strategy[lap] = tyre

	best_position = result.fun

	print(f"Best Strategy: {best_strategy}")
	print(f"Best Finishing Position: {best_position}")

	return best_strategy, best_position

# Example usage
# best_strategy, best_position = optimize_strategy_up_to_3_stops(race_data, driver_number=10, n_calls=100)
# print(f"Best Strategy: {best_strategy}")
# print(f"Best Finishing Position: {best_position}")

In [20]:
from bayes_opt import BayesianOptimization
import numpy as np

def bayesian_strategy_optimization(race_data, given_driver, max_iterations=50):
	"""
	Optimize the pit strategy for a given driver using Bayesian Optimization.
	
	Args:
		race_data (dict): Precomputed race data.
		given_driver (int): The driver number to optimize the strategy for.
		max_iterations (int): Maximum number of iterations for optimization.
		
	Returns:
		tuple: Best strategy (dict) and best finishing position (int).
	"""
	# Extract precomputed data
	max_laps = race_data["max_laps"]
	initial_strategy = race_data["driver_strategies"][given_driver]  # Use the driver's actual race strategy

	# Define the objective function for Bayesian Optimization
	def objective_function(starting_tyre, pit1_lap, pit1_tyre, pit2_lap, pit2_tyre, pit3_lap, pit3_tyre):
		# Ensure pit laps are unique and within valid range
		pit_laps = sorted(set([int(pit1_lap), int(pit2_lap), int(pit3_lap)]))
		tyres = [int(pit1_tyre), int(pit2_tyre), int(pit3_tyre)]
		
		# Filter out invalid pit laps (e.g., out of bounds or overlapping)
		valid_pit_laps = []
		valid_tyres = []
		for lap, tyre in zip(pit_laps, tyres):
			if 2 <= lap < max_laps:  # Only include valid pit laps
				valid_pit_laps.append(lap)
				valid_tyres.append(tyre)

		# Construct the strategy dictionary
		strategy = {1: int(starting_tyre)}  # Starting tyre
		for lap, tyre in zip(valid_pit_laps, valid_tyres):
			strategy[lap] = tyre

		# Penalize strategies with fewer than 2 distinct tyre types
		tyre_types_used = set(strategy.values())
		if len(tyre_types_used) < 2:
			return -20.0  # Penalize with worst position (negative because we're maximizing)

		# Evaluate the strategy using the race simulation
		try:
			sim_df = race_sim(race_data, given_driver=given_driver, simulated_strategy=strategy)
			final_position = sim_df[sim_df["driver_number"] == given_driver]["position"].iloc[-1]
			return -final_position  # Negative because BayesianOptimization maximizes by default
		except Exception as e:
			print(f"Error during simulation: {e}")
			return -20.0  # Return worst position in case of errors

	# Set up the parameter bounds for Bayesian Optimization
	pbounds = {
		"starting_tyre": (1, 3),  # Starting tyre (1=Hard, 2=Medium, 3=Soft)
		"pit1_lap": (2, max_laps - 1),  # First pit stop lap (optional)
		"pit1_tyre": (1, 3),  # First pit stop tyre (optional)
		"pit2_lap": (2, max_laps - 1),  # Second pit stop lap (optional)
		"pit2_tyre": (1, 3),  # Second pit stop tyre (optional)
		"pit3_lap": (2, max_laps - 1),  # Third pit stop lap (optional)
		"pit3_tyre": (1, 3),  # Third pit stop tyre (optional)
	}

	# Initialize the Bayesian Optimizer
	optimizer = BayesianOptimization(
		f=objective_function,
		pbounds=pbounds,
		verbose=2,
		random_state=42
	)

	# Perform the optimization
	optimizer.maximize(init_points=5, n_iter=max_iterations)

	# Extract the best strategy from the optimizer
	best_params = optimizer.max["params"]
	best_starting_tyre = int(best_params["starting_tyre"])
	pit_laps = sorted(set([int(best_params["pit1_lap"]), int(best_params["pit2_lap"]), int(best_params["pit3_lap"])]))
	tyres = [int(best_params["pit1_tyre"]), int(best_params["pit2_tyre"]), int(best_params["pit3_tyre"])]

	# Construct the best strategy dictionary
	best_strategy = {1: best_starting_tyre}
	for lap, tyre in zip(pit_laps, tyres):
		if 2 <= lap < max_laps:  # Only include valid pit laps
			best_strategy[lap] = tyre

	# Evaluate the best strategy to get the finishing position
	sim_df = race_sim(race_data, given_driver=given_driver, simulated_strategy=best_strategy)
	best_position = sim_df[sim_df["driver_number"] == given_driver]["position"].iloc[-1]

	print(f"Best Strategy: {best_strategy}")
	print(f"Best Finishing Position: {best_position}")

	return best_strategy, best_position

# Example Usage
best_strategy, best_position = bayesian_strategy_optimization(
	race_data=race_data,
	given_driver=11,
	max_iterations=20
)


|   iter    |  target   | pit1_lap  | pit1_tyre | pit2_lap  | pit2_tyre | pit3_lap  | pit3_tyre | starti... |
-------------------------------------------------------------------------------------------------------------
{np.int64(1): {np.int64(1): {np.int64(1): [0.0009178771155909023, -0.0033808734085401214, 1.4812535875778954], np.int64(2): [0.00025020590751303367, 0.04678985440191217, 2.3323948143526527], np.int64(3): [0.0003769096949929787, 0.03620968255451015, 0.2515056771404921]}, np.int64(3): {np.int64(1): [0.0015917968379534134, -0.0024937994940423523, 1.3380719741247395], np.int64(2): [0.00060309742596996, 0.04449313587652371, 2.2154982890751547], np.int64(3): [0.0011323805205723763, 0.01480476070882824, 0.3366559121434837]}, 2: {}}, np.int64(3): {np.int64(1): {np.int64(1): [0.005293122930733136, -0.07420533349290309, 1.536182530937377], np.int64(2): [0.001979200966503208, 0.0056913308661828465, 2.64661407244786], np.int64(3): [0.00034201746775766595, 0.03283228341506487, 0.559

KeyError: 2